## importing packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
chunksize = 2048

# load file
import os
import soundfile as sf
# read file
fileDir = os.path.dirname(os.path.realpath('__file__'))
file_name = os.path.join(fileDir, '../sounds/ttls u3.wav')
file_name = os.path.abspath(os.path.realpath(file_name))
ref, sample_rate = sf.read(file_name)

t_start = 1.63
pos_start = int(t_start*44100)
pos_end = pos_start + chunksize*4
signal = ref[pos_start: pos_end, 0]

# add noise so that silent parts will not give ambiguous values
# signal = np.add(signal, 0.001*np.random.randn(len(signal)))
# sd.play(signal, sample_rate)

In [3]:
# to be called at the end of every 2048 entries
# the input is 2048 entry * 3
def detect_onset(signal, chunksize=2048, tempo_res=32):
    onset = -1
    
    difference = np.cumsum(np.add(np.absolute(signal[chunksize:-chunksize]), -np.absolute(signal[:-2*chunksize])))
    noise = 10*np.array(np.random.randn(len(difference)))
    difference = np.add(difference, noise)
    
    roceff = np.full(tempo_res, 0.)
    tempo_num = int(chunksize / tempo_res)
    for i in range(tempo_res):
        roceff[i] = np.corrcoef(difference[i * tempo_num:(i * tempo_num + chunksize)],
                                      np.arange(chunksize))[0, 1]
        if roceff[0] < 0.8 and roceff[i] > 0.8 and np.max(roceff[:i]) < 0.8:
            onset = i
    
#     plt.figure(figsize = (16,2))
#     plt.plot(signal)
#     plt.plot(np.arange(chunksize, chunksize*3),np.array(difference)/np.max(np.absolute(difference)))
#     plt.plot(np.arange(chunksize*2, chunksize*3, tempo_num),roceff)
#     plt.axhline(y=0.8)
#     plt.show()
    
    return onset # none, or a value

In [4]:
detect_onset(signal)


14